# Documentation of Economic Analysis behind Simulation Engine - Part 1

In this notebook we describe the logic behind the economic model that feeds into the economic simulation engine of Emergent Alliance's _Regional Risk-Pulse Index_ project. The economic model that we base our approach upon is Wassily Leontief's Input-Output model, which was first proposed in 1951 and which has become a common tool for economists to understand economic networks (see Carvalho and Tahbaz-Salehi (2018) [1], for instance). At its core, the model is based on the idea that sectors in an economy are interconnected through input-output linkages. In other words, part of what one sector produces is used as an input by other sectors. The building block of the Input-Output model is the so-called Leontief's Inverse Matrix. To see how Leontief's inverse matrix is relevant, note that the $(i; j)$ element of the Leontief inverse, $l_{ij}$, measures the importance of industry $j$ as a direct and indirect input-supplier to industry $i$ in the economy. Furthermore, $l_{ij}$ accounts for all possible directed walks (of various lengths) that connect industry $j$ to industry $i$ over the network.

In what follows, we construct the Leontief inverse matrix using the worldwide IO tables provided by the [OECD](https://www.oecd.org/sti/ind/inter-country-input-output-tables.htm) [2]. We use the data for the last year available, 2015, published in 2018. The process to build this matrix based on the raw data consists of these steps: 
- First, we take the matrix of trade flows between sectors (the raw table that we get from the OECD), which we call Z, and aggregate all non-essential sectors. 
- Second, we construct the column vector x, which represents the total production by sector. 
- Third, we combine x and Z to get matrix A, which gives us the coefficients IO table. The difference between A and Z is that Z represents trade flows, whereas A represents the input linkages in terms of relative proportions (assuming a Cobb-Douglas production function). 
- Finally, we construct the Leontief inverse L from A. The Leontief inverse is defined as $L = (I-A)^{-1}$.

We will use a package called `pymrio` to ingest and process the input-output tables. You can read the docs [here](https://pymrio.readthedocs.io/en/latest/).

To know more about these matrices, their meaning, the Leontief model and how it is used in general, this [paper](http://vasco-m-carvalho.github.io/pdfs/ProductionNetworks.pdf) by Carvalho and Tahbaz-Salehi (2018) is a good reference. The notation for A and L is the same. Z is referred to as trade flows matrix. x is not explicitly mentioned.




**Table of contents**  
* [Input-output flows matrix Z](#Z)
* [Final demands vector x](#x)
* [Coefficients matrix A](#A)
* [Leontief's inverse matrix L](#L)

**Inputs**
- OECD's Input-Output tables [1]

**Outputs** 
- All matrices relevant to Leontief's Input-Output model

In [1]:
# Imports and path
import warnings
try:
    import pymrio
except:
    warnings.warn('Pymrio missing')
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import networkx as nx
import seaborn as sbs
import random


%matplotlib inline
oecd_storage = Path('/project_data/data_asset')

/opt/conda/envs/Python-3.6-WMLCE/lib/python3.6/site-packages/ipykernel/__main__.py:6: UserWarning: Pymrio missing


In [3]:
# Select the year to parse - 2015 in this case
oecd_path_year = pymrio.parse_oecd(path=oecd_storage, year=2015)

<a id='Z'></a>
### Input-Output flows matrix Z
Firs, we read the input-output tables and format them to get the matrix representing the direct relations between the sectors. We'll call this matrix Z. There are some non-essential sectors that we are not interested in, so we aggregate them under the label `All non-essential`. A sector has been classified as non-essential in line with the Italian government's [decree](http://www.governo.it/sites/new.governo.it/files/dpcm_20200322.pdf) from the 22nd of March.

In [ ]:
countries = oecd_path_year.get_regions()
non_essential_sectors = ['07T08','24','25','29','30','68']

In [ ]:
# Get Input-Output table
Z = oecd_path_year.Z.copy()
Z.shape

In [ ]:
# Collapse non-essential columns
country_NE = pd.DataFrame()

for country in countries:
    
    for sector in non_essential_sectors:
        
        column = Z[(country, sector)]
        country_NE = pd.concat([country_NE, column], axis = 1)
        Z.drop(columns = (country, sector), inplace = True)
    
    country_NE = country_NE.sum(axis = 1)
    
    Z[(country, 'All non-essential')] = country_NE
    
    country_NE = pd.DataFrame()

In [ ]:
# Collapse non-essential rows
country_NE = pd.DataFrame()

for country in countries:
    
    for sector in non_essential_sectors:
        
        row = Z.loc[(country, sector),:]
        country_NE = pd.concat([country_NE, row], axis = 1)
        Z.drop(index = (country, sector), inplace = True)
    
    country_NE = country_NE.sum(axis = 1)
    
    Z.loc[(country, 'All non-essential'),:] = country_NE
    
    country_NE = pd.DataFrame()

Z

<a id='x'></a>
### Final demands vector x
The package pymrio has a method to construct x. It needs the industries' final demands, Y, as well as Z. We need to collapse the final demand from non-essential sectors before combining Z and Y.

In [ ]:
Y =  oecd_path_year.Y.copy()
Y.shape

In [ ]:
# Collapse non-essential rows
country_NE = pd.DataFrame()

for country in countries:
    
    for sector in non_essential_sectors:
        
        row = Y.loc[(country, sector),:]
        country_NE = pd.concat([country_NE, row], axis = 1)
        Y.drop(index = (country, sector), inplace = True)
    
    country_NE = country_NE.sum(axis = 1)
    
    Y.loc[(country, 'All non-essential'),:] = country_NE
    
    country_NE = pd.DataFrame()
    
Y

We can now combine matrices Z and Y to produce the total outputs vector x. We simply need to call pymrio's method `calc_x`.

In [ ]:
x = pymrio.calc_x(Z, Y)
x

<a id='A'></a>
### Coefficients matrix A
Once we have x and Z, we can move on to calculate A.

In [ ]:
A = pymrio.calc_A(Z, x)
A

<a id='L'></a>
### Leontief's inverse matrix L

Finally, having A we can derive the Leontief inverse matrix. 

In [ ]:
L = pymrio.calc_L(A)
L

In [ ]:
# Let's have a look at one country only
L.iloc[L.index.get_level_values('region') == 'DEU',
       L.columns.get_level_values('region') == 'DEU']

The matrices are now ready to be exported and used elsewhere. We will continue using these in this notebook to illustrate the other pieces of work.

In [ ]:
# Export to CSV
# L.to_csv('/project_data/data_asset/Leontief_inverse_world.csv')
# Y.to_csv('/project_data/data_asset/Leontief_final_demand_Y_world.csv')
# x.to_csv('/project_data/data_asset/Leontief_total_output_x_world.csv')
# A.to_csv('/project_data/data_asset/Leontief_coeff_A_world.csv')

#### Authors
* **Álvaro Corrales Cano** is a Data Scientist within IBM's Cloud Pak Acceleration team. With a background in Economics, Álvaro specialises in a wide array Econometric techniques and causal inference, including regression, discrete choice models, time series and duration analysis.
* **Deepak Shankar Srinivasan** is a Developer in R2 Data Labs, Rolls Royce Deutschland, Germany, specializing in Data Science applications for Equipment Health Management and Deep Domain Specific Smart Assistants.




Copyright © IBM Corp. 2020. Licensed under the Apache License, Version 2.0. Released as licensed Sample Materials.

________

[1] Carvalho, V.M., Tahbaz-Salehi, A (2018). Production Networks: A Primer. Working paper (Forthcoming in The Annual Review of Economics). Retrieved from http://vasco-m-carvalho.github.io/pdfs/ProductionNetworks.pdf

[2] OECD (2018), OECD Inter-Country Input-Output (ICIO) Tables, URL: https://www.oecd.org/sti/ind/inter-country-input-output-tables.htm, last accessed: 26 June 2020